<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных</a></span></li><li><span><a href="#Подготовка-данных-для-обучения" data-toc-modified-id="Подготовка-данных-для-обучения-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных для обучения</a></span></li><li><span><a href="#Обучение,-валидация" data-toc-modified-id="Обучение,-валидация-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение, валидация</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Тест-моделей" data-toc-modified-id="Тест-моделей-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тест моделей</a></span></li></ul></div>

# Переход на новый тарифный план

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами.

**Цель**: проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф, с максимально большим значением accuracy (0.75 или больше). Проверим accuracy на тестовой выборке.

## Изучение данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime as dt
import seaborn as sns
import os
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import warnings

In [2]:
#warnings.simplefilter("once")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
pth1 = '/datasets/users_behavior.csv'
pth2 = 'users_behavior.csv'
if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    print('Something is wrong')

In [4]:
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


В таблице 5 столбцов, 3214 строк. Пропусков данных нет. Предобработка и исследовательский анализ данных не требуются, т.к. были сделаны в ранее другом проекте.

* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца (***«Ультра» — 1, «Смарт» — 0***).

## Подготовка данных для обучения
Разобьём выборку на признаки, целевой признак, а также на тренировочную, валидационную и тестовую выборки.
<br>В нашем случае целевой признак - это `is_ultra`. А задача - бинарная классификация, т.к. тарифов всего 2.

In [94]:
features = df.drop(columns=['is_ultra'])
target = df['is_ultra']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, 
    train_size=0.8, shuffle=True, random_state=12345
)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, 
    test_size=0.5, shuffle=True, random_state=12345
)

print(len(features_train))
print(len(features_valid))
print(len(features_test))
display(features_train.head(1))
display(features_valid.head(1))
display(features_test.head(1))

2571
321
322


,calls,minutes,messages,mb_used
348,77.0,522.65,14.0,19462.45


,calls,minutes,messages,mb_used
1552,105.0,724.57,158.0,39930.07


,calls,minutes,messages,mb_used
76,73.0,633.22,21.0,18421.17


## Обучение, валидация 

### Дерево решений

In [95]:
best_tree_model = None
best_accuracy = 0
best_tree_depth = 0

In [96]:
for max_depth in range(1,11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=max_depth)
    model.fit(features_train, target_train)
    accuracy = model.score(X=features_valid, y=target_valid) 
    print('max_depth =', max_depth, ':', accuracy)
    if (accuracy > best_accuracy):
        best_accuracy = accuracy
        best_tree_model = model
        best_tree_depth = max_depth
        
print('\nBest decision tree: depth=', best_tree_depth, ', accuracy=', best_accuracy, sep='')

max_depth = 1 : 0.7414330218068536
max_depth = 2 : 0.7819314641744548
max_depth = 3 : 0.7881619937694704
max_depth = 4 : 0.7881619937694704
max_depth = 5 : 0.794392523364486
max_depth = 6 : 0.7975077881619937
max_depth = 7 : 0.7912772585669782
max_depth = 8 : 0.7850467289719626
max_depth = 9 : 0.7912772585669782
max_depth = 10 : 0.7757009345794392

Best decision tree: depth=6, accuracy=0.7975077881619937


### Случайный лес

In [97]:
best_forest_model = None
best_accuracy = 0
best_forst_depth = 0
best_forest_n = 0

In [98]:
for est in range(10, 51, 3):
    for depth in range (2, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        accuracy = model.score(features_valid, target_valid)
        if accuracy > best_accuracy:
            best_forest_model = model
            best_accuracy = accuracy
            best_forest_depth = depth
            best_forest_n = est

print('Best random forest: depth=', best_forest_depth, ', n_estimators=', best_forest_n, ', accuracy=', best_accuracy, sep='')

Best random forest: depth=8, n_estimators=43, accuracy=0.8068535825545171


### Логистическая регрессия

In [99]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)

print('Logistic regression: accuracy=', accuracy, sep='')

Logistic regression: accuracy=0.67601246105919


## Тест моделей

In [100]:
accuracy = best_tree_model.score(features_test, target_test)
print('Decision tree: depth=', best_tree_depth, ', accuracy=', accuracy, sep='')

accuracy = best_forest_model.score(features_test, target_test)
print('Random forest: depth=', best_forest_depth, ', n_estimators=', best_forest_n, ', accuracy=', accuracy, sep='')

accuracy = model.score(features_test, target_test)
print('Logistic regression: accuracy=', accuracy, sep='')

Decision tree: depth=6, accuracy=0.7670807453416149
Random forest: depth=8, n_estimators=43, accuracy=0.8012422360248447
Logistic regression: accuracy=0.7298136645962733


In [101]:
dt.datetime.today().strftime("%d.%m.%Y %H:%M")

'18.04.2022 23:51'